# Magics 4.13.0 ( Release June 2022)

In the release:  
- Wind components defined in 2 different files 
- introduction of a series of out_of_bond parameters 
- Adjustement of the symbol height according to the projection extend. (symbol size will increase when zooming)
 

## Wind components defined in 2 differents files


In [ ]:
import os
from ecmwfapi import ECMWFService

mars = {
        "class": "od",
        "date": -1,
        "levelist": "500",
        "levtype": "pl",
        "param": ["u"],
        "stream": "oper",
        "time": "00",
        "type": "fc",
        "grid" : [5,5]
 }
if not os.path.exists("data/u.grib"):
    server = ECMWFService("mars")
    mars["param"] = ["u"]
    server.execute(
        mars, 
        "data/u.grib"
    )
if not os.path.exists("data/v.grib"):
    server = ECMWFService("mars")
    mars["param"] = ["v"]
    server.execute(
        mars, 
        "data/v.grib"
    )
if not os.path.exists("data/tu.grib"):
    server = ECMWFService("mars")
    mars["param"] = ["t", "u"]
    server.execute(
        mars,
        "data/tu.grib"
    )

if not os.path.exists("data/zv.grib"):
    server = ECMWFService("mars")
    mars["param"] = ["z", "v"]
    server.execute(
        mars, 
        "data/zv.grib"
    )
if not os.path.exists("data/uvt.grib"):
    server = ECMWFService("mars")
    mars["param"] = ["u", "v", "t"]
    server.execute(
        mars,
        "data/uvt.grib"
    )

if not os.path.exists("data/tzuv.grib"):
    server = ECMWFService("mars")
    mars["param"] = ["t", "u", "z", "v"]
    server.execute(
        mars,
        "data/tzuv.grib"
    )
    
if not os.path.exists("data/multiuv_ml.grib"):
    server = ECMWFService("mars")
    mars["param"] = ["u", "v"]
    mars["levelist"] = [10, 20, 30, 40]
    mars["levtype"] = "ml"
    server.execute(
        mars,
        "data/multiuv_ml.grib"
    )
            

In [ ]:
from Magics import macro as magics
import ipywidgets as widgets
from ipywidgets import interact

print(magics.version())

In [ ]:
!grib_ls data/uvt.grib

In [ ]:

coast = magics.mcoast()
title = magics.mtext( text_lines = ["wind from uv.grib"], text_colour = "charcoal")

def plot_uv():
    wind = magics.mgrib(grib_input_file_name = "data/uvt.grib",
                       grib_wind_position_1 = 2,
                       grib_wind_position_2 = 3)
    flags = magics.mwind(wind_field_type = "flags", wind_flag_colour="navy")
    return wind, flags, "wind from uv.grib in navy"

def plot_u_v():
    wind = magics.mgrib(grib_first_component_file_name = "data/u.grib", 
                       grib_second_component_file_name = "data/v.grib",
                       )
    flags = magics.mwind(wind_field_type = "flags", wind_flag_colour="red")
    return wind, flags, "wind from u.grib and v.grib in red"

def plot_u2_v2():
    wind = magics.mgrib(grib_first_component_file_name = "data/tu.grib", 
                       grib_second_component_file_name = "data/zv.grib",
                       grib_wind_position_1 = 2,
                       grib_wind_position_2 = 2,
                       )
    flags = magics.mwind(wind_field_type = "flags", wind_flag_colour="evergreen")
    return wind, flags, "wind from tu.grib and tv.grib in evergreen"

def plot_u3_v4():
    wind = magics.mgrib(grib_first_component_file_name = "data/tzuv.grib", 
                       grib_second_component_file_name = "data/tzuv.grib",
                       grib_wind_position_1 = 3,
                       grib_wind_position_2 = 4,
                       )
    flags = magics.mwind(wind_field_type = "flags", wind_flag_colour="pink")
    return wind, flags, "wind from tzuv.grib and tzuv.grib in pink"
    
    


@interact(test = widgets.Dropdown(description='test', options = [("test1", plot_u_v), 
                                                                 ("test2", plot_u2_v2), 
                                                                 ( "test3", plot_u3_v4)]),)

def compare(test):
    text = []
    uv, uv_flags, uv_text = plot_uv()
    u_v, u_v_flags, u_v_text = test()  
    text.append(uv_text)
    text.append(u_v_text)
    title = magics.mtext( text_lines = text, text_colour = "charcoal")
    return magics.plot(uv, uv_flags, coast, u_v, u_v_flags, title)
    
    

# Coloured wind 

In [ ]:
wind = magics.mgrib(grib_input_file_name = "data/uvt.grib", 
                    grib_wind_position_1 = 2,
                    grib_wind_position_2 = 3,
                    grib_wind_position_colour = 1)
wind_split = magics.mgrib(grib_first_component_file_name = "data/tu.grib", 
                       grib_second_component_file_name = "data/zv.grib",
                       grib_colour_component_file_name = "data/tu.grib",
                       grib_wind_position_1 = 2,
                       grib_wind_position_2 = 2,
                       grib_wind_position_colour = 1)
flags = magics.mwind(legend  = 'on',
    wind_field_type                          = 'flags',
    wind_advanced_method                     = 'on',
    wind_advanced_colour_selection_type      = 'interval',
    wind_advanced_colour_level_interval      = 1.0,
    wind_advanced_colour_reference_level     = 20.0,
    wind_advanced_colour_parameter = "parameter",
    wind_advanced_colour_table_colour_method = 'calculate',
    wind_advanced_colour_direction           = 'clockwise',
    wind_advanced_colour_min_level_colour    = 'blue',
    wind_advanced_colour_max_level_colour    = 'red')

magics.plot(wind_split, flags)

# Missing data, NetCDF  and graph

Sometimes in can be difficult to handle mssing values and automatic setting of the cartesian projection, in particular when trying to display a tinme serie.  

We introduce a new parameter **netcdf_ignore_missing_value** that will give you a bit more control for time series.  
If **True** the default,  we will  ignore the missing values ans setup the date axis to only show the non-missing values.  
if **False**, we will extend the time series and include the missing values.  
Try it below to get a bettre understanding.




@interact( netcdf_ignore_missing_value = widgets.ToggleButtons(
    options=['on', 'off'],
    description='netcdf_ignore_missing_value:',
    tooltips=['Will ignore the msising values', 'Will ignore the msising values'],
))

def plot(netcdf_ignore_missing_value):
#Setting the cartesian view
    projection = magics.mmap( 
                subpage_map_projection = 'cartesian',
                subpage_x_axis_type = 'date',
                subpage_y_axis_type = 'regular',
                subpage_x_automatic="on",
                subpage_y_automatic="on",           
                # subpage_x_date_min = x[0],
                # subpage_x_date_max = x[-1],
                )

#Vertical axis
    vertical = magics.maxis(axis_orientation = "vertical",
                     axis_type = "regular",
                     axis_tick_label_height = 0.4,
                     axis_tick_label_colour = 'navy',
                     axis_grid =  "off",
                     axis_highlighted_values = [0.045, 0.052],
                     axis_highlighted_values_colour = "evergreen",
                     axis_highlighted_values_style = "dash",
                     axis_highlighted_values_thickness = 10,
                    )

#Horizontal axis
    horizontal = magics.maxis(axis_orientation = "horizontal",
                     axis_type = "date",
                     axis_grid =  "on",
                     axis_days_label_height = 0.40,
                     axis_months_label_height = 0.40,
                     axis_years_label_height = 0.50,
                     axis_grid_colour = "grey",
                     axis_grid_thickness = 1,
                     axis_grid_line_style = "dot")





    data = magics.mnetcdf (
        netcdf_dimension_setting= ["flag:Used", "area:Globe"],
        netcdf_filename= "data/graph-data.nc",
        netcdf_type= "xypoint",
        netcdf_x_variable= "time",
        netcdf_y_variable= "andep_bcor",
        netcdf_ignore_missing_value = netcdf_ignore_missing_value
    )
    graph = magics.mgraph(graph_line_thickness=10 )



    title = magics.mtext(
               text_lines = ["netcdf_ignore_missing_value: {} ".format(netcdf_ignore_missing_value)],
               text_justification = "left",
               text_font_size = 1.,
               text_colour =  "charcoal")


#To the plot
    return magics.plot(projection,   data, graph, vertical, horizontal,  title)

# out of bond


Expected behaviour:  
- **contour_level_selection_type=interval**:  
interval respected between **contour_out_of_bound_min** and **contour_out_of_bound_max** using contour_reference as reference
- **contour_level_selection_type=list**:  
level list respected between **contour_out_of_bound_min** and **contour_out_of_bound_max**  
- **contour_level_selection_type=count**:  
~n levels between  **contour_out_of_bound_min** an  **contour_out_of_bound_max**


In [7]:
from Magics import macro as magics
import ipywidgets as widgets
from ipywidgets import interact

x = [i for i in range(-40, 40, 6)]
print (x)

@interact(contour_level_selection_type=widgets.Dropdown(description='contour_level_selection_type',
                                                options= [ "interval", "count", 
                                                "level_list"], value='level_list'))
def plot(contour_level_selection_type):

    data = magics.mgrib(grib_input_file_name = "data/uvt.grib")
    contour = magics.mcont(
            legend                       = 'on',
            contour                      = 'off',
            contour_shade                = 'on',
            contour_label                = 'off',
            contour_level_selection_type = contour_level_selection_type,
            contour_level_list = [i for i in range(-40, 40,6)], 
            contour_interval             = 5,
            contour_level_tolerance = 0,
            contour_level_count = 10,
            contour_shade_method         = 'area_fill',
            contour_shade_palette_name   = "m_yellow_red_16",
            contour_shade_colour_method  = "palette",
            contour_shade_colour_list_policy = "dynamic",
            contour_out_of_bound_min= -35,
            contour_out_of_bound_max= 5,
            contour_out_of_bound_min_colour = "green",
            contour_out_of_bound_max_colour = "yellow",
            # contour_shade_min_level = -28,
            # contour_shade_max_level = -10,
            # contour_min_level = -45,
            # contour_max_level = -2,
        )
    legend = magics.mlegend(legend_display_type = "continuous")
    coast = magics.mcoast(map_coastline_resolution= "medium")
    return magics.plot(data, contour, coast, legend) 

[-40, -34, -28, -22, -16, -10, -4, 2, 8, 14, 20, 26, 32, 38]


interactive(children=(Dropdown(description='contour_level_selection_type', index=2, options=('interval', 'coun…

# Get legend metadata


In [ ]:

metadata = magics.mmap(metadata_path = "legend.json")

data = magics.mgrib(grib_input_file_name = "data/uvt.grib")
contour = magics.mcont(
        legend                       = 'on',
        contour                      = 'off',
        contour_shade                = 'on',
        contour_label                = 'off',
        contour_level_selection_type = "interval",
        contour_interval             = 5,
        contour_shade_method         = 'area_fill',
        contour_shade_palette_name   = "m_yellow_red_16",
        contour_shade_colour_method  = "palette",
        contour_shade_colour_list_policy = "dynamic",
        contour_out_of_bound_min= -40,
        contour_out_of_bound_min_colour = "evergreen",
        contour_out_of_bound_max= -5,
        contour_out_of_bound_max_colour = "navy",
    )
legend = magics.mlegend(legend_display_type = "continuous")
coast = magics.mcoast(map_coastline_resolution= "medium")
magics.plot(metadata, data, contour, coast, legend) 


# Boxplot examples 


In [1]:
from Magics import macro as magics

from datetime import datetime, timedelta
import random

start = datetime.now()
dates = []
maximum = []
minimum = []
median = []
lower = []
upper = []

for i in range(0, 246, 6):
    date = start+ timedelta(hours=i)
    dates.append(date.strftime("%Y-%m-%d %H:00"))
    maximum.append(random.randrange(10, 20))
    minimum.append(random.randrange(-20, -10))
    median.append(random.randrange(-5, 5))
    lower.append(random.randrange(-10, -5))
    upper.append(random.randrange(5, 10))
    
    


In [2]:
import xmltodict

import ipywidgets as widgets
from ipywidgets import interact, VBox

with open("/users/Sylvie/git/magics-develop/src/params/BoxPlotVisualiser.xml") as fd:
    l = fd.read()
    definition = xmltodict.parse(l)

def boolean(data):
    name = data["@name"]
    return widgets.ToggleButtons(description=name, options = ["on", "off"], 
                                 style = {'description_width': 'initial', 'width': 'max-content'})

def colour(data):
    name = data["@name"]
    return widgets.Dropdown(description=name, options = ["red", "blue", "green", "navy", "pink", "sky"],
                            value=data["@default"],
                            style = {'description_width': 'initial', 'width': 'max-content'})

def style(data):
    name = data["@name"]
    return widgets.Dropdown(description=name, options = ["dash", "solid", "dot"], value=data["@default"],
                           style = {'description_width': 'initial', 'width': 'max-content'})


def choice(data):
    name = data["@name"]
    values = data.get("@values", None)
    if values:
        return widgets.Dropdown(description=name, options =values.split("/"),  value=data["@default"],
                               style = {'description_width': 'initial', 'width': 'max-content'})
    return ""


def ignore(data):
    name = data["@name"]
    return None

def float(data):
    name = data["@name"]
    return widgets.FloatSlider(description=name,  min= 0.1, max =5, value=1,
                               style = {'description_width': 'initial', 'width': 'max-content'})

def int(data):
    name = data["@name"]
    min = 0
    max = 100
    if "thickness" in name:
        min =1
        max = 5
    return widgets.IntSlider(description=name,  min= 1, max =5, value=1,
                               style = {'description_width': 'initial', 'width': 'max-content'})

helpers = {
    "bool" : boolean,
    "Colour" : colour,
    "float" : float,
    "int" : int,
    "LineStyle" : style,
    "string" : choice,
}


all = {}

for e in definition["magics"]["class"]["parameter"]:
    
    to = e["@to"]
    w = helpers.get(to, ignore)(e)
    if w : 
        all[e["@name"]] = w
        

            




In [4]:
import ipywidgets as widgets
from ipywidgets import interactive_output, VBox
from Magics import macro as magics


def plot(**args):
    projection = magics.mmap(subpage_y_min = -20.00,
                    subpage_map_projection = "cartesian",
                    subpage_y_position = 2.00,
                    subpage_y_max = 20.00,
                    subpage_x_axis_type = "date",
                    subpage_y_axis_type = "regular",
                    subpage_x_date_min = dates[0],
                    subpage_x_date_max =  dates[-1],)

    vertical = magics.maxis(axis_orientation = "vertical",
                    axis_grid_reference_line_style = "solid",
                    axis_grid_reference_thickness = 2,
                    axis_grid_thickness = 1,
                    axis_grid_reference_level = 0.00,
                    axis_type = "regular",
                    axis_grid = "on",
                    axis_grid_line_style = "dot",
                    axis_grid_colour = "grey")

    horizontal = magics.maxis(axis_orientation = "horizontal",
                    axis_grid_thickness = 1,
                    axis_type = "date",
                    axis_days_label_height = 0.40,
                    axis_grid = "on",
                    axis_months_label_height = 0.40,
                    axis_years_label_height = 0.40,
                    axis_grid_line_style = "dot",
                    axis_grid_colour = "grey")
    
    args["boxplot_date_positions"] = dates
    args["boxplot_maximum_values"] = maximum
    args["boxplot_median_values"] = median 
    args["boxplot_minimum_values"] = minimum
    args["boxplot_box_lower_values"] = lower
    args["boxplot_box_upper_values"] = upper
    data = magics.mboxplot(
                           args
                          
                          )



    display(magics.plot(projection, vertical, horizontal, data))
    
ws = []
for w in all:
    ws.append(all[w])
    
ui = widgets.VBox(ws)
out = interactive_output(plot, all)

display(ui, out)

Output()